In [1]:
import os
import time
import datetime

import pandas as pd
import seaborn as sns
import numpy as np
import random

import matplotlib.pyplot as plt


import torch
from torch.utils.data import Dataset, DataLoader, random_split, RandomSampler, SequentialSampler
from nltk.translate.bleu_score import sentence_bleu
torch.manual_seed(42)

from transformers import GPT2LMHeadModel,  GPT2Tokenizer, GPT2Config, GPT2LMHeadModel
from transformers import AdamW, get_linear_schedule_with_warmup

In [2]:
import string, re
# test_set = pd.read_csv('medialog_eng.csv')
# print(f"meddialog shape - {test_set.shape}")
# regex = re.compile('[%s]' % re.escape(string.punctuation))

# lasse_df = pd.read_csv('lasse_qa.csv')
# print(f"lasse_qa shape - {lasse_df.shape}")
# lasse_test_df = lasse_df.sample(n=test_set.shape[0])
# test_set = test_set.append(lasse_test_df)
# print(f"test_set size {test_set.shape}")

# test_set.to_csv('test_set.csv')

test_set = pd.read_csv('test_set.csv')

def process_string(answer):
    p_answer = answer.translate(str.maketrans('','', string.punctuation))
    curr_ans = p_answer.split(' ')
    return curr_ans
    

In [3]:
gpt_tokenizer = GPT2Tokenizer.from_pretrained('gpt2', bos_token='<|startoftext|>', eos_token='<|endoftext|>', pad_token='<|pad|>') #gpt2-medium

gpt_path = 'C:/Users/wjtay/Documents/GitHub/medicalbot/gpt_models/model_save_3_lasseOnly_30'
gpt_model =  GPT2LMHeadModel.from_pretrained(gpt_path)
gpt_model.resize_token_embeddings(len(gpt_tokenizer))

gpt_model.eval()




Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


GPT2LMHeadModel(
  (transformer): GPT2Model(
    (wte): Embedding(50259, 768)
    (wpe): Embedding(1024, 768)
    (drop): Dropout(p=0.1, inplace=False)
    (h): ModuleList(
      (0): GPT2Block(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2Attention(
          (c_attn): Conv1D()
          (c_proj): Conv1D()
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (mlp): GPT2MLP(
          (c_fc): Conv1D()
          (c_proj): Conv1D()
          (dropout): Dropout(p=0.1, inplace=False)
        )
      )
      (1): GPT2Block(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2Attention(
          (c_attn): Conv1D()
          (c_proj): Conv1D()
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )


In [4]:


def get_prediction(n, query):
    prompt = f"<|startoftext|><|question|>{query}<|answer|>"

    generated = torch.tensor(gpt_tokenizer.encode(prompt)).unsqueeze(0)
    # generated = generated.to(device)

    # print(generated)

    sample_outputs = gpt_model.generate(
                                    generated, 
                                    #bos_token_id=random.randint(1,30000),
                                    do_sample=True,   
                                    top_k=50, 
                                    max_length = 300,
                                    top_p=0.95, 
                                    num_return_sequences=n
                                    )
    decoded_outputs = []

    for i, sample_output in enumerate(sample_outputs):
        # print("{}: {}\n\n".format(i, tokenizer.decode(sample_output, skip_special_tokens=True)))
        output = gpt_tokenizer.decode(sample_output, skip_special_tokens=True)
        output = output.split('<|answer|>')[1]
        decoded_outputs.append(output)
    
    
    # print(decoded_outputs)
    return decoded_outputs


In [5]:
from datasets import load_metric
metric = load_metric("bleurt")

Using default BLEURT-Base checkpoint for sequence maximum length 128. You can use a bigger model for better results with e.g.: datasets.load_metric('bleurt', 'bleurt-large-512').


INFO:tensorflow:Reading checkpoint C:\Users\wjtay\.cache\huggingface\metrics\bleurt\default\downloads\extracted\3f355b8dbe6b9f8d2e5d3e510191b6db066af8d9a08e9d2d81314d2e3ff24161\bleurt-base-128.
INFO:tensorflow:Config file found, reading.
INFO:tensorflow:Will load checkpoint bert_custom
INFO:tensorflow:Loads full paths and checks that files exists.
INFO:tensorflow:... name:bert_custom
INFO:tensorflow:... vocab_file:vocab.txt
INFO:tensorflow:... bert_config_file:bert_config.json
INFO:tensorflow:... do_lower_case:True
INFO:tensorflow:... max_seq_length:128
INFO:tensorflow:Creating BLEURT scorer.
INFO:tensorflow:Creating WordPiece tokenizer.
INFO:tensorflow:WordPiece tokenizer instantiated.
INFO:tensorflow:Creating Eager Mode predictor.
INFO:tensorflow:Loading model.
Instructions for updating:
If using Keras pass *_constraint arguments to layers.
INFO:tensorflow:BLEURT initialized.


In [14]:
for idx in range(5):
    row = test_set.iloc[idx]
    references = [row['answer']]
    predictions = get_prediction(1, row['question'])
    print(f"predictions {predictions} ")
    print(f"references {references} ")
    metric.add(prediction=predictions, reference=references)
    # metric.add(prediction=["hello world"], reference=["hello worlds"])
    # print(metric.compute(prediction=["hello world"], reference=["hello worlds"]))
    

score = metric.compute()

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


predictions ['most helpful answer that would not be possible. the vaccine is not 100% either. there are many skin eruptions that can mimic a sore and sometimes cause a type of infection. you are needed to have this properly evaluated. unfortunately it would not be possible to blindly determine the nature of this infection based solely on your vague description. the medical provider who is doing the test is in the best position to make that call.'] 
references ["during this pandemic. throat pain can be from a strep throat infection (antibiotics needed), a cold or influenza or other virus, or from some other cause such as allergies or irritants. usually, a person sees the doctor (call first) if the sore throat is bothersome, recurrent, or doesn't go away quickly. covid-19 infections tend to have cough, whereas strep throat usually lacks cough but has more throat pain. (3/21/20)"] 


Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


predictions ['if you also have an unusual cough it would be wise to see your medical provider -- someone who knows you and your medical history -- first. this could be a sign of an infection and you should get it checked out. as you know cold and influenza are viral infections. one virus in particular tends to cause higher fevers but not one that often happens right after a cold or even a virus. note of the three common symptoms that can be caused by colds: sore throat body aches extreme tiredness and weakness. fever with no other cold symptoms. it would be best to get the checked out sooner rather than later. i am a type 2 diabetic so finding out if you have an infection may be helpful.'] 
references ['yes. protection. it is not enough symptoms to say that you are a suspect case of covid19; but, independently of this, if you have been in contact with a case, or you present persistent cough (with or without sputum), shortness of breath, wheezing, or you have a chronic disease like diab

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


predictions ['if your symptoms persist it would be best to be seen by a physician soon. there is no way to determine a specific cause of your symptoms without a hands-on medical examination and some diagnostic tests.'] 
references ['possible. top symptoms include fever, dry cough and sob. an obvious possibility. if so, your best step is to self-quarntine. remember at your age low risk of complication and typically will pass without issue. if worsening sob be seen. call your provider or check with local health department. these are healthtap guidelines:https://www.healthtap.com/blog/covid-19-care-guidelines/self-quarantine-guide.'] 


Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


predictions ['lactid 23 is the upper and end of the chromosomesomalase mutations. there will be ainge of hives and redness at this time.'] 
references ['in brief: symptoms if you are infected, symptoms will emerge: tiredness, dry cough, fever worsening over 5-14 days. you will also become more infective so self-isolation and good hygiene are vital.only be concerned about covid-19 if: - you have been in contact with someone with a conformed diagnosis of covid-19 - you have visited a high risk area - symptoms worsen and include persistent fever and dry cough would you like to video or text chat with me?'] 
predictions ["hi its the miscarriage it could be from the bed rest it could be from the hiccup in your hormones that's giving you your pain and its him giving you this lift again the same with your pregnancy. if not see your doctor for a check up this would tell his/her all you need. hope this helps. good luck"] 
references ['thanks for your question on healthcare magic.i can understan

In [13]:
# import pickle
# def save_obj(obj, filepath ):
#     with open(filepath + '.pkl', 'wb') as f:
#         pickle.dump(obj, f, pickle.HIGHEST_PROTOCOL)

# def load_obj(filepath ):
#     with open(filepath + '.pkl', 'rb') as f:
#         return pickle.load(f)

print(score)
# print(sum(score['scores']) / len(score['scores']))


{'scores': [-0.8576737642288208, -1.0169175863265991, -0.9189336895942688, -1.621059536933899, -1.1398746967315674]}


In [16]:
print(test_set.iloc[4]['question'])

just found out i was pregnant. yesterday diagnosed with pneumonia. i am a high risk pregnancy. fertility issues, pcos, weak cervix. delivered first daughter at 29 weeks, miscarried, and gave birth at 38 weeks to second daughter, but was on bedrest for weak cervix beginning at 5 months. i m a wreck. when i miscarried they said my progesterone level is low which caused me to miscarry, and gave me progesterone shots every week. can t see doctor for two days.


In [14]:
for root, dirs, files in os.walk('C:/Users/wjtay/Documents/GitHub/medicalbot/gpt_models'):
    print(os.path.join(root,dirs))

TypeError: join() argument must be str, bytes, or os.PathLike object, not 'list'

## Old Pipeline

In [17]:
from transformers import BertTokenizer, BertForMaskedLM, BertModel
import torch
import pandas as pd
from tqdm import tqdm

# BERT_MODEL = "microsoft/BiomedNLP-PubMedBERT-base-uncased-abstract-fulltext"
BERT_MODEL = "bert-base-uncased"
DAT_PATH = "lasse_qa.csv"
OUTPUT_NAME = "med_bert_lasse_only.csv"
tokenizer = BertTokenizer.from_pretrained(BERT_MODEL)

def create_mbert_tokens(title):
    # print(f"preparing embedding for {title}")
    tokens = {'input_ids': [], 'attention_mask': []}
    new_tokens = tokenizer.encode_plus(title, max_length=90, truncation=True,
                                            padding='max_length', return_tensors='pt')

    # tokens['input_ids'] = new_tokens['input_ids'][0]
    # tokens['attention_mask'] = new_tokens['attention_mask'][0]
    tokens['input_ids'].append(new_tokens['input_ids'][0])
    tokens['attention_mask'].append(new_tokens['attention_mask'][0])

    # reformat list of tensors into single tensor
    tokens['input_ids'] = torch.stack(tokens['input_ids'])
    tokens['attention_mask'] = torch.stack(tokens['attention_mask'])
    # print(tokens["attention_mask"])
    return tokens['input_ids'], tokens['attention_mask']

def bert_embed_gen(title, model):
    model = model.bert
    tokens = create_mbert_tokens(title)
    token_mapping = {
        "input_ids": tokens[0],
        "attention_mask": tokens[1]
    }
    # print(tokens)
    # passing the tokens into model to get the pre-trained embeddings
    outputs = model(**token_mapping)
    # print(outputs)
    # outputs = self.model(new_tokens['input_ids'][0], attention_mask=new_tokens['attention_mask'][0])
    embeddings = outputs.last_hidden_state
    # print(embeddings)

    # resizing the attention mask to the correct dimensions
    attention_mask = token_mapping['attention_mask']
    mask = attention_mask.unsqueeze(-1).expand(embeddings.size()).float()

    # applying the mask onto the embeddings
    masked_embeddings = embeddings * mask

    # converting the multuple token embeddings into a single embedding (ie sentence embedding)
    # this is done by taking the mean of the different token embeddings
    summed = torch.sum(masked_embeddings, 1)
    summed_mask = torch.clamp(mask.sum(1), min=1e-9)
    mean_pooled = summed / summed_mask

    mean_pooled = mean_pooled.detach().numpy()
    
    return mean_pooled

# question model
question_model = BertForMaskedLM.from_pretrained(BERT_MODEL)
question_model.eval()

# answer model
answer_model = BertForMaskedLM.from_pretrained(BERT_MODEL)
answer_model.eval()


Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForMaskedLM: ['cls.seq_relationship.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForMaskedLM: ['cls.seq_relationship.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model

BertForMaskedLM(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=Tr

In [19]:
from sklearn.metrics.pairwise import cosine_similarity

curr_model = 'lasseOnly_30'
results_df = pd.DataFrame(columns=['question', 'pred_ans', 'actual_ans', 'pred_ans_emb', 'actual_ans_emb', 'cos_dist'])

for idx in tqdm(range(test_set.shape[0])):
    row = test_set.iloc[idx]
    question = row["question"]
    actual_ans_emb = bert_embed_gen(row["answer"], answer_model)
    answer = process_string(row["answer"])
    predicteds = get_prediction(3, question)

    for output in predicteds:
        # print(output)
        predicted = output.split('<|answer|>')[1]
        predicted_ans_emb = bert_embed_gen(predicted, answer_model)
        datapoint = {}
        datapoint.update({'question' : question})
        datapoint.update({'pred_ans' : predicted})
        datapoint.update({'actual_ans' : row["answer"]})
        datapoint.update({'pred_ans_emb' : predicted_ans_emb})
        datapoint.update({'actual_ans_emb' : actual_ans_emb})
        datapoint.update({'cos_dist' : cosine_similarity(actual_ans_emb, predicted_ans_emb)[0][0]})
        results_df = results_df.append(datapoint, ignore_index = True)
        
    break

display(results_df)
results_df.to_csv(f'C:/Users/wjtay/Documents/GitHub/medicalbot/test_data/{curr_model}.csv')


  0%|          | 0/604 [00:09<?, ?it/s]


,question,pred_ans,actual_ans,pred_ans_emb,actual_ans_emb,cos_dist
0,throat a bit sore and want to get a good imune...,most helpful answer that would be the virus th...,during this pandemic. throat pain can be from ...,"[[-0.1396009, 0.1651383, -0.028681934, -0.0555...","[[-0.29372224, 0.06123257, 0.24927963, -0.2278...",0.749834
1,throat a bit sore and want to get a good imune...,most helpful answer that would not be the reco...,during this pandemic. throat pain can be from ...,"[[-0.18819077, 0.12383803, -0.0024491036, -0.2...","[[-0.29372224, 0.06123257, 0.24927963, -0.2278...",0.840972
2,throat a bit sore and want to get a good imune...,most helpful answer this is a viral infection ...,during this pandemic. throat pain can be from ...,"[[-0.093472265, -0.052979365, 0.052858457, -0....","[[-0.29372224, 0.06123257, 0.24927963, -0.2278...",0.774828


In [20]:
results_df['pred_ans'][1]

'most helpful answer that would not be the recommended dosage for an injection nor should you be self-treating. a visual exam is vital before proceeding. the medical provider who is treating you should not have seen this strain of sore throat. cold air is not the initial sign of an infection so perhaps a tiny bit of an infection is not really present.'

In [30]:
from nltk.translate.bleu_score import sentence_bleu
all_bleu_score = []

for idx, row in test_set.iterrows():
    question = row["question"]
    answer = process_string(row["answer"])
    predicteds = get_prediction(3, question)

    curr_bleu_score = 0
    for output in predicteds:
        predict = output.split('<|answer|>')[1]
        curr_bleu_score = max(curr_bleu_score, sentence_bleu(answer, process_string(predict)))
        print(f'bleu: {curr_bleu_score}')
        # print(f'ans: {answer}')
        # print(f'predict: {process_string(predict)}')
    
    all_bleu_score.append(curr_bleu_score)


Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
C:\Python38\lib\site-packages\nltk\translate\bleu_score.py:515: UserWarning: 
The hypothesis contains 0 counts of 2-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)
C:\Python38\lib\site-packages\nltk\translate\bleu_score.py:515: UserWarning: 
The hypothesis contains 0 counts of 3-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)
C:\Python38\lib\site-packages\nltk\translate\bleu_score.py:515: UserWarning: 
The hypothesis contains 0 counts of 4-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or

bleu: 7.752349855101862e-232
bleu: 7.850707993042515e-232
bleu: 7.850707993042515e-232


Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


bleu: 7.544641259221007e-232
bleu: 7.544641259221007e-232
bleu: 7.544641259221007e-232


Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


...

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


bleu: 8.087494814426946e-232
bleu: 8.087494814426946e-232
bleu: 8.301113029505914e-232


Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


bleu: 7.817228731469945e-232
bleu: 7.817228731469945e-232
bleu: 7.817228731469945e-232


KeyboardInterrupt: 